This is the same thing as getWhissell but using the emotion dictionary from Warriner at al 2013. They have many more words (almost 14,000) but no function words, and they use lemmas on the assumption that only a few plurals are used more often than singulars and as such should not have very distinct emotional profiles. So processing this dictionary for scores will be a bit different. Their three vectors are Valence, Arousal, and Dominance. Dominance is not related to Whissell's Imagery so that third score will have to be neglected for now (though they discuss imageability on p. 1199. Brysbaert et al 2014 provides concreteness scores, and it is there that they discuss choosing lemmas over other forms. For that list, which they don't say how it might relate to this one, they include plurals that are used as different parts of speech or that are used more frequently than the singular [eyes]). So as a first guess lets lemmatize the text, not run a stop list but look up each word and keep it on the Nonelist for now. Lemmatization has its own pitfalls so we need to keep an eye on that. Valence (EE) mean is 5.06, Activation (AA) is 4.21, and Dominance (DD) is 5.18 (on a scale from 1-9, lowest-highest). I will try using SpaCy instead of NLTK, for speed and so it will scale. 

In [82]:
import spacy
import textacy
import re

targetDoc = "texts/waves-par-lemmas.txt"     # the text to analyze (relative path), already lemmatized
targetWord = "light"               # the word type you want to score
targetWindow = 5                  # this is tokens before and after target
#outFile = "texts/mdg-lemmas.txt"

#file = open(outFile, "w")

document = open(targetDoc).read().split()
numbered = enumerate(document)

hitlist = []               #make a list of hits
for i, j in numbered:
    if j == targetWord:
        hitlist.append(i)
print(hitlist)        
for hit in hitlist:
    phrase = []
    for index, word in enumerate(document):   # why can't we reuse numbered? This can't be the best way
        if index >= (hit - targetWindow) and index <=  (hit + targetWindow):
            phrase.append(word)
    print(phrase)
        
 
      
#file.close()

#print(hitlist)
#for hit in hitlist:
#    phrase = []                  #keep a list of tokens in the phrase
#    for j in range(hit-targetWindow, hit+targetWindow):
#        if spaceDoc[j].is_alpha and spaceDoc[j].lemma_ != "-PRON-":
#            #print(spaceDoc[j].lemma_)
#            phrase.append(spaceDoc[j].lemma_)
#    
#    print(" ".join(word for word in phrase))

           


[38, 95]
['bernard', 'hang', 'quiver', 'hang', 'loop', 'light', 'slab', 'pale', 'yellow', 'say', 'susan']
['bernard', 'bead', 'water', 'drop', 'white', 'light', 'pass', 'house', 'house', 'like', 'friar']


In [ ]:
#nlp = spacy.load("en")           #ok this works to get the ords and such. But there might be a better method.
#spaceDoc = nlp(document)
#hitlist = []                #make a list of hits
#for t in spaceDoc:                   # t.is_alpha = not punctuation, t.sent = word's sentence
#    if t.text == targetWord:
#        hitlist.append(t.i)
#print(hitlist)
#for hit in hitlist:
#    phrase = []                  #keep a list of tokens in the phrase
#    for j in range(hit-targetWindow, hit+targetWindow):
#        if spaceDoc[j].is_alpha and spaceDoc[j].lemma_ != "-PRON-":
#            #print(spaceDoc[j].lemma_)
#            phrase.append(spaceDoc[j].lemma_)
#    
#    print(" ".join(word for word in phrase))

    spaCy is a text processing package in Python. It slurps up the text and does lot sof things to it, one thing being lemmatizations. textacy is a wrapper for spaCy that makes it simple to access lots of the data that Spacy makes. As you can see above, several things can be accomplished by a single line. Working on the whole novel The Waves, it takes a bout 15 seconds and some windup on the hard drive to do anything, so its a bit slow. On the other hand, it gets the stuff out in pretty good form. I installed spacy using Anaconda, then had to do textacy from the prompt using conda. The respective program documentations describes how to do that. 

In [59]:
quotes = textacy.extract.direct_quotations(doc)  #gets a rough list of quotations. Good first pass.
for quote in quotes:
    print(quote)


(Rainsford, asked, "What island is it?")
(Rainsford, asked, "The old charts call it 'Ship-Trap Island,'")
(imagination, said, "Pure imagination,")
(imagination, said, "One superstitious sailor can taint the whole ship's company with his fear.")
(he, thought, "It's so dark,")
(he, thought, "Where there are pistol shots, there are men. Where there are men, there is food,")
(he, said, "It is a very great pleasure and honor to welcome Mr. Sanger Rainsford, the celebrated hunter, to my home.")
(he, said, "So am I.")
(he, said, "Come,")
(he, said, "we shouldn't be chatting here. We can talk later. Now you want clothes, food, rest. You shall have them. This is a most -- restful spot.")
(Follow Ivan, said, "Follow Ivan, if you please, Mr. Rainsford,")
(Follow Ivan, said, "I was about to have my dinner when you came. I'll wait for you. You'll find that my clothes will fit you, I think.")
(he, suggested, "You'll have a cocktail, Mr. Rainsford,")
(Half, said, "We do our best to preserve the ameni

In [5]:
for sent in doc.sents:    #prints sentences that have the target word in them
    for word in sent:
        if word.text == targetWord:
            print(sent)

It quivers and
hangs in a loop of light.  
It has beads of water on it, drops of white light.  


This last command below creates a set of subj verb obj triples for each sentence. The catch is that it works only on transitive sentences. It's pretty good though. Ideally one would have it run on intransitives and transitives, then also on embedded clauses and possible even nonfinite clauses for a complete view of agency. But again its a good easy start. I should add that these textacy functions all create what Python calls a "generator". This is essentially an iterator without the iteration statement. The generator "yields" one item at a time, so it can be processed before the next item is presented. For example, the trips variable gives up one trip per go. The for-loop gets each trip in turn, does something to it, then gets the next one. In this case I am just printing, but we can do other things like make lists of all the transitive verbs or ones that take particular subjects, etc.


In [ ]:
trips = textacy.extract.subject_verb_object_triples(doc) # makes a generator; transitives only
for item in trips:
    print(item)

So the catch with getting lemmas is that to get the lemmas spacy needs the parse data, and to get that it needs the sentences, which of course need punctuation to detect. So we have to leave the punctuation in, run it into spacy, then find the target words, get their indexes, count forward and backward _excluding_ punctuation, get the lemmas for those indexes, then do the math.